# Elasticsearch Setup Fix

This notebook helps fix the Elasticsearch connection issues when running the notebooks locally.

## 1. Set Environment Variables

First, we need to set the correct environment variables to connect to Elasticsearch. The Docker Compose file maps Elasticsearch port 9200 to host port 9201, so we need to use that port when connecting from the host machine.

In [ ]:
import os

# Set environment variables for local testing
os.environ["AIPS_ES_HOST"] = "localhost"  # Use localhost when running outside Docker
os.environ["AIPS_ES_PORT"] = "9201"       # This matches the host port in docker-compose.yml

## 2. Test Elasticsearch Connection

Now let's test the connection to Elasticsearch.

In [ ]:
import requests

# Test Elasticsearch connection
es_url = f"http://{os.environ['AIPS_ES_HOST']}:{os.environ['AIPS_ES_PORT']}"
try:
    response = requests.get(f"{es_url}/_cluster/health")
    if response.status_code == 200:
        print(f"✅ Successfully connected to Elasticsearch at {es_url}")
        print(f"Cluster status: {response.json()['status']}")
    else:
        print(f"❌ Failed to connect to Elasticsearch at {es_url}")
        print(f"Status code: {response.status_code}")
except Exception as e:
    print(f"❌ Error connecting to Elasticsearch at {es_url}: {e}")

## 3. Import AIPS and Set Engine

Now we can import the AIPS library and set the engine to Elasticsearch.

In [ ]:
from aips import get_engine, set_engine

# Set the engine to Elasticsearch
set_engine("elasticsearch")
engine = get_engine()

# Check if the engine is working
if engine.health_check():
    print("✅ Elasticsearch engine is healthy")
else:
    print("❌ Elasticsearch engine is not healthy")

## 4. Run the Original Notebook

Now you can run the original notebook (ch04/1.setting-up-the-retrotech-dataset.ipynb) with these environment variables set. You can either:

1. Copy the environment variable settings to the beginning of that notebook, or
2. Run this notebook first, then run the original notebook in the same Jupyter session.

Here's a quick test to create a collection and add a document:

In [ ]:
# Create a test collection
try:
    collection = engine.create_collection("test_collection")
    print("✅ Successfully created test_collection")
    
    # Add a test document
    doc = {"id": "1", "title": "Test Document", "content": "This is a test document"}
    collection.add_documents([doc])
    print("✅ Successfully added test document")
    
    # Search for the document
    response = collection.search(query="test")
    print(f"✅ Search results: {response}")
except Exception as e:
    print(f"❌ Error testing collection: {e}")

## 5. Troubleshooting

If you're still having issues, check the following:

1. Make sure Elasticsearch is running in Docker:
   ```bash
   docker compose ps
   ```

2. If it's not running, start it:
   ```bash
   docker compose up elasticsearch
   ```

3. Check the Elasticsearch logs:
   ```bash
   docker compose logs elasticsearch
   ```

4. Make sure the port mapping is correct in docker-compose.yml:
   ```yaml
   ports:
     - 9201:9200
   ```

5. Try accessing Elasticsearch directly in your browser: http://localhost:9201